In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import scipy

In [2]:
import sys
sys.path.append('../Preprocess')
import CalcParaMaps
import CTPPreprocess

In [3]:
def ReadImg(filename, iSlices, nx = 256, ny = 256):
    with open(filename, 'rb') as f:
        f.seek(nx*ny*iSlices[0]*4)
        img = np.fromfile(f, np.float32, nx*ny*len(iSlices)).reshape([len(iSlices), nx, ny])
    
    return img

In [4]:
inputDir = '/home/dwu/trainData/Noise2Noise/train/ctp/phantom/'
outputDir = '/home/dwu/trainData/Noise2Noise/train/ctp/simul/data/'
if not os.path.exists(outputDir):
    os.makedirs(outputDir)

In [5]:
nx = 256
ny = 256
iSlices = np.arange(100, 170)
iAifSlice = 49
nFrames = 50

In [6]:
# read all the images
imgs = []
for i in range(nFrames):
    img = ReadImg(os.path.join(inputDir, str(i+1)), iSlices)
    imgs.append(img[...,np.newaxis])
imgs = np.concatenate(imgs, -1)
np.save(os.path.join(outputDir, 'refs.npy'), imgs)

In [7]:
# read all the paametric ground truth
cbf0 = ReadImg(os.path.join(inputDir, 'cbf'), iSlices)
cbv0 = ReadImg(os.path.join(inputDir, 'cbv'), iSlices)
mtt0 = ReadImg(os.path.join(inputDir, 'mtt'), iSlices)
baseline0 = ReadImg(os.path.join(inputDir, 'baseline'), iSlices)
np.savez(os.path.join(outputDir, 'paras0'), cbf=cbf0, cbv=cbv0, mtt=mtt0, baseline=baseline0)

In [8]:
# get the aif
ctps = imgs - imgs[...,[0]]
mask = np.where(ctps[...,25] > 0, 1, 0)
aif, _ = CTPPreprocess.GetVOF(ctps[[iAifSlice], ..., :])
aif[aif < 0] = 0
np.save(os.path.join(outputDir, 'aif0'), aif)

In [12]:
# calculate all the parametric maps
cbf, cbv, mtt = CalcParaMaps.CalcParaMaps(ctps, mask, method='tikh', lam = 0.3, kappa=1, rho=1, 
                                          aif = np.copy(aif), useCircConv=True, N=None)

In [13]:
facMask = np.where(ctps[...,25] * mask < 9, 1, 0) * mask
cbfFac = np.sum(cbf0 * facMask) / np.sum(cbf * facMask)
print (cbfFac)

cbvFac = np.sum(cbv0 * facMask) / np.sum(cbv * facMask)
print (cbvFac)

cbf *= cbfFac
cbv *= cbvFac
mtt = mtt / cbfFac * cbvFac

2.7970992833304646
1.1315869383762456


In [15]:
# save parametric maps and correction factor
np.savez(os.path.join(outputDir, 'paras_tikh_0.3'), cbf=cbf, cbv=cbv, mtt=mtt, mask=mask, cbfFac=cbfFac, cbvFac = cbvFac)